# Make tables: calculate p-values and differential expressions for each cancer 

Create a dataframe with p-value results from t-tests for all proteins (trans proteomics when PTEN has cnv deletions compared to PTEN wildtype). The dataframe also includes the change in medians between deletions and wildtype. Prepare these tables for further analysis by creating csv files.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats
import re

import cptac
import cptac.utils as u
import plot_utils as p

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Returns a dataframe with proteomics and mutation type

# all_prot: list of trans genes

def all_prot_format_df(cancer_object, all_prot, gene_in = 'PTEN', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    
    if cancer_object.get_cancer_type() not in ('luad'):
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(
            mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = all_prot, tissue_type = 'tumor')
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
            prot_and_mutations = rename_duplicate_cols(prot_and_mutations)
        
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] 
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type)
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    elif cancer_object.get_cancer_type() in ('luad'): 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns= gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = cancer_object.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1= gene_in, 
            genes2= all_prot, tissue_type = 'tumor')
        omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = rename_duplicate_cols(omics)
        omics = omics.drop(columns='PTEN_CNV')
       
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type) 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
        mut_wt['Mutation'] = np.where(
            mut_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')
    
    mut_wt = mut_wt.rename(columns={col: re.sub(r'_proteomics', '', col) for col in mut_wt.columns.tolist()})
    return mut_wt


In [4]:
'''
Params
cancer_object: Object. Variable for the loaded cancer data set.
cancer_name: Str. name to add to the created dataframe.
del_wt_df: DataFrame. Only samples with deletions and wildtype for PTEN. 
all_prot_list: List. All proteins in proteomics data frame. 

Returns a dataframe with the difference in medians between proteomics with PTEN wt and del (wt - del). 
'''

def get_change_in_medians_df(cancer_object, cancer_name, del_wt_df, all_prot_list):
    if  cancer_object.get_cancer_type() in ('endometrial'):
        d = del_wt_df[del_wt_df.Mutation == "Truncation"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    else:
        d = del_wt_df[del_wt_df.Mutation == "Deletion"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    del_med = d.median()
    wt_med = wt.median()

    med_dict = {}
    
    # Correlation: + is mutant up compared to wt, - is mutant down
    for prot in all_prot_list:
        if prot in del_med.index and prot in wt_med.index:
            dif = del_med[prot] - wt_med[prot]
            med_dict[prot] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Proteomics'})
    
    return df
        

In [5]:
# Adds '_i' to duplicate col names where i is a number (increases with each duplicate)

def rename_duplicate_cols(df):
    cols = pd.Series(df.columns[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols
    return df

#  Step 1: Create data frames with p-values and differential expressions

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run t-tests for all genes in the proteomic data frame. Use get_change_in_medians_df to create the data frame with differential expression values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [6]:
g = cptac.Gbm()
en = cptac.Endometrial()
h = cptac.Hnscc()
l = cptac.Luad()
ls = cptac.Lscc()
o = cptac.Ovarian()
col = cptac.Colon()
b = cptac.Brca()

In [147]:
pten_int = u.get_interacting_proteins_wikipathways('PTEN')
len(pten_int)

1328

In [183]:
gene = 'PTEN'
cancer_obj = {'Colon': col, 'LSCC': ls, 'BR': b, 'OV': o} 

for name in cancer_obj:
    cancer = cancer_obj[name]
    print(name)
    prot = cancer.get_proteomics()
    if isinstance(prot.keys(), pd.core.indexes.multi.MultiIndex):
        prot= u.reduce_multiindex(prot, levels_to_drop = 1)
    prot = prot.drop(columns = 'PTEN') # cis effect
    prot_cols = list(prot.columns)
    
    # Choose how many proteins to test
    #prot_list = pten_int # interacting
    prot_list = prot_cols # all proteins
    
    del_wt = all_prot_format_df(cancer, prot_list)

    # check how many samples are in stage 1 and have PTEN loss
    clinical = cancer.get_clinical(tissue_type = 'tumor')


    # get a list of all samples with initial stages
    id_list = []

    if cancer.get_cancer_type() in ('colon'):
        stage_list = ['Stage I', 'Stage II'] # colon
    if cancer.get_cancer_type() in ('lscc'):
        stage_list = ['I',  'IA', 'IB'] # LSCC
    if cancer.get_cancer_type() in ('brca'):
        stage_list = ['Stage IIA', 'Stage IIB'] #brca
    if cancer.get_cancer_type() in ('ovarian'):
        stage_list = ['IV'] #ov

    for stage in stage_list:
        if cancer.get_cancer_type() in ('ovarian'):
            stage_df = clinical.loc[clinical['Tumor_Stage_Ovary_FIGO'] == stage]
        else:
            stage_df = clinical.loc[clinical['Stage'] == stage]

        s1 = list(stage_df.index)
        id_list += s1
    #print('Num total samples of initial stages:', len(id_list))

    # Keep samples of initial stages
    initial = del_wt[del_wt.index.isin(id_list)]
    print(initial.Mutation.value_counts())

    prot_cols = list(initial.columns[:-1])

    # Get all pvals
    pval = u.wrap_ttest(initial, 'Mutation', prot_cols, return_all = True, correction_method = 'fdr_bh', mincount = 3)
    print(pval, '\n')

Colon
Wildtype_Tumor    36
Deletion          16
Name: Mutation, dtype: int64
     Comparison   P_Value
0          NAXE  0.095506
1        FAM45A  0.175503
2          FHL2  0.338536
3       NDUFAF3  0.338536
4      EPB41L4B  0.381870
...         ...       ...
7144      RPS19  0.999189
7145      RPS4X  0.999189
7146     KIF13A  0.999189
7147    PGLYRP2  0.999371
7148      TEX10  0.999901

[7149 rows x 2 columns] 

LSCC
Deletion          17
Wildtype_Tumor    16
Name: Mutation, dtype: int64
      Comparison   P_Value
0          PARVA  0.080529
1         CAVIN3  0.080529
2         ZSWIM8  0.080529
3          FBXO2  0.080529
4         MYOF_1  0.080529
...          ...       ...
11549       UFC1  0.998698
11550       ESPN  0.998698
11551     CORO1A  0.998774
11552     ZNF581  0.999530
11553     TUBB4B  0.999743

[11554 rows x 2 columns] 

BR
Wildtype_Tumor    42
Deletion          18
Name: Mutation, dtype: int64
      Comparison   P_Value
0           TIA1  0.144369
1         ANP32A  0.144369
2